## Ceramid Biosynthesis
Absence of Ceramids in discovered when reconstructing GSM. gene function loss in the pathway? potential target for genetical engineering

-> **Metabolic network reconstruction of Chlamydomonas offers insight into light-driven algal metabolism (Chang et al. 2011)**

**Approach**: Used the Hetero model.  I added to the our start product Palmitoyl-Coa Reactions to produce our Final product N-Acylsphingosine(Ceramide)
- for metabolites and reactions the KEGG ID was used 

**How to continue**: 
- add an exchange reaction?
- what changes in the metabolism of c.reinhardtii? - can we find the growth rate?
- find a way to increase palmitoyl-coa production/ or decrease palmitoy-coa degradation(beta-oxidation reaction transport in to the glykosom: FA160COAabcp), chagnes in medium, find genes?
- compare with the mixo and auto model 


In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import cobra.util.solver as sutil
from cobra.flux_analysis import flux_variability_analysis as fva
from cobra.exceptions import OptimizationError
import cobra.test
from cobra.flux_analysis import production_envelope
from cobra.flux_analysis.phenotype_phase_plane import production_envelope, add_envelope
from os.path import dirname, join, pardir
from cobra.io import read_sbml_model

In [2]:
from cobra.io import read_sbml_model
hetero = read_sbml_model('iCre1355_hetero.xml')

Model does not contain SBML fbc package information.
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre10.g451950.t1.2 OR Cre06.g284700.t1.2 )' for '<Reaction AAT at 0x7f7c4a101438>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre02.g143200.t1.1 OR ( Cre08.g368900.t1.2 OR Cre08.g368900.t2.1 ) )' for '<Reaction ALATLm at 0x7f7c4a1019b0>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre09.g390986.t1.1 OR Cre06.g279150.t1.2 )' for '<Reaction ASPTLm at 0x7f7c4a111e48>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre17.g747297.t1.1 OR Cre02.g076600.t1.2 )' for '<Reaction ALATAH at 0x7f7c4a11f320>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre17.g747297.t1.1 OR Cre02.g076600.t1.2 )' for '<Reaction ASNTAH at 0x7f7c4a11f5c0>'
cobra/core/reaction.py:440 UserWarning: uppercase AND/OR found in rule '( Cre17.g747297.t1.1 OR Cre02.g076600.t1.2 )' for '<

### Needs to be corrected:
- find acyl_coa  and add to reaction
- add 3Dehydrosphinganine to reaction

### Ceramid biosynthesis: genome.jp/kegg-bin/show_module?M00094+C00154

**Included in our model already**: C00154(Palmitoyl-CoA), C02934 (3-Dehydrosphinganine)

In [4]:
hetero.metabolites.get_by_id('pmtcoa_c')

Metabolite identifier,pmtcoa_c
Name,Palmitoyl-CoA (n-C16:0CoA)
Memory address,0x07f7c4a34cb70
Formula,C37H62N7O17P3S
Compartment,c
In 19 reaction(s),"ACOADAGAT16018111Z160, SPHGNAT, SERPT, AGPATCOA1819Z160, ACOADAGAT18111Z18111Z160, ACOADAGAT1819Z1819Z160, SPHINGSAT, G3PAT160, 3OACOAS, ACOADAGAT1801819Z160, ACOADAGAT1819Z18111Z160, PMTCOAth, FAC..."


In [3]:
# Palmitoyl -CoA
hetero.metabolites.pmtcoa_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,PMTCOAtm,100.0,0.012767,pmtcoa_m <=> pmtcoa_c
CONSUMING,FA160COAabcp,100.0,0.012767,atp_c + h2o_c + pmtcoa_c --> adp_c + h_c + pi_...


### Reaction R01281
Palmitoyl-Coa + L-Serine <-> 3-Dehydropsphinganine + CoA + CO2

In [7]:
hetero.reactions.get_by_id('SERPT')

Reaction identifier,SERPT
Name,serine C-palmitoyltransferase
Memory address,0x07f7d02395048
Stoichiometry,h_c + pmtcoa_c + ser_L_c --> 3dsphgn_c + co2_c + coa_c H+ + Palmitoyl-CoA (n-C16:0CoA) + L-Serine --> 3-Dehydrosphinganine + CO2 + Coenzyme A
GPR,( Cre03.g146507.t1.1 and Cre04.g230144.t1.1 )
Lower bound,0.0
Upper bound,1000.0


In [6]:
hetero.reactions.SERPT.summary()

In [8]:
### issue can´t get info about 3-Dehydrosphinganine because starts with integer

**Further pathway**: C00836 (Sphinganine) -> C12126(Dihydroceramide) -> C00195(N-Acylsphingosine)

## R02978 = 3-Dehydrosphinganine (3dsphgn_c) + NADPH + H+ <-> Sphinganine (C00836) + NADP+

In [19]:
R02978 = Reaction('R02978')

In [20]:
sphinganine = Metabolite(id='c00836_c', compartment = 'c')

In [21]:
R02978.add_metabolites({hetero.metabolites.h_c: -1,
                              #hetero.metabolites.3dsphgn_c: -1,
                            hetero.metabolites.nadph_c: -1,
                              sphinganine: 1, 
                              hetero.metabolites.nadp_c: 1
                             })

In [23]:
print(R02978.build_reaction_string())

h_c + nadph_c --> c00836_c + nadp_c


In [25]:
hetero.add_reactions([R02978])

## R06517: Acyl-CoA + Sphinganine (c00836_c) <-> CoA (coa_c) + Dihydroceramide

In [26]:
R06517 = Reaction('R06517')

In [27]:
dihydroceramide = Metabolite(id='c12126_c', compartment = 'c')

In [29]:
R06517.add_metabolites({#hetero.metabolites.acyl_coa: -1, have to find acyl coa
                              hetero.metabolites.c00836_c: -1,
                            hetero.metabolites.coa_c: 1,
                              dihydroceramide: 1
                             })

In [30]:
print(R06517.build_reaction_string())

c00836_c --> c12126_c + coa_c


In [31]:
hetero.add_reactions([R06517])

## R06519: Dihydroceramide (c00836_c) + 2 Ferrocytochrome b5 (focytb5_c) + Oxygen (o2_c) + 2H (h_c) <-> N-Acylsphingosine + 2 Ferricytochrome b5 (ficytb5_c) + 2H20 (h2o_c)

In [33]:
R06519 = Reaction ('R06519')

In [34]:
nacylsphingosine = Metabolite(id='c00195_c', compartment = 'c')

In [36]:
R06519.add_metabolites({
    hetero.metabolites.c00836_c: -1,
    hetero.metabolites.focytb5_c: -2,
    hetero.metabolites.o2_c: -1,
    hetero.metabolites.h_c: -2,
    nacylsphingosine: 1,
    hetero.metabolites.ficytb5_c: 2,
    hetero.metabolites.h2o_c: 2
                             })

In [38]:
print(R06519.build_reaction_string())

c00836_c + 2 focytb5_c + 2 h_c + o2_c --> c00195_c + 2 ficytb5_c + 2 h2o_c


In [39]:
hetero.add_reactions([R06519])